# Notebook to calculate average and median CB & LB voltages on the SGT Fleet
## From voltage at 200 minutes

## Load BDT files from folder and store as dataframes in a list

In [10]:
import os
import pandas as pd
import numpy as np

# Init main dir and list
main_directory = 'BDT_fleet_latest' 
latest_fleet_bdt = []
cb_terminal_voltage = ['RT1BMCBBV', 'RT4BMCBBV', 'RT5BMCBBV', 'RT8BMCBBV']
lb_terminal_voltage = ['RT1BMLBBV','RM2BMLBBV','RM3BMLBBV','RT4BMLBBV','RT5BMLBBV','RM6BMLBBV','RM7BMLBBV','RT8BMLBBV']

file_path = []

# Look through folder
for file_name in os.listdir(main_directory):
    if file_name.endswith('.csv'):
        temp_file_path = os.path.join(main_directory, file_name)
        file_path.append(temp_file_path)
        file_path.sort()
        
# Read the CSV file into a df and put in list
for i, path in enumerate(file_path):
    df = pd.read_csv(file_path[i])
    latest_fleet_bdt.append(df)

# Check that files are loaded correctly
# and are sorted
print(file_path)
# print(latest_fleet_bdt[0])

['BDT_fleet_latest/B01_CSVLog-2024-04-23_2344.csv', 'BDT_fleet_latest/B02_CSVLog-2024-10-16_0333.csv', 'BDT_fleet_latest/B03_CSVLog-2024-09-09_0108.csv', 'BDT_fleet_latest/B04_CSVLog-2024-09-10_1359.csv', 'BDT_fleet_latest/B05_CSVLog-2024-08-14_0326.csv', 'BDT_fleet_latest/B06_CSVLog-2024-07-31_0335.csv', 'BDT_fleet_latest/B07_CSVLog-2024-02-29_2155.csv', 'BDT_fleet_latest/B08_CSVLog-2024-08-26_0221.csv', 'BDT_fleet_latest/B09_CSVLog-2024-09-18_0244.csv', 'BDT_fleet_latest/B10_CSVLog-2024-09-27_0244.csv', 'BDT_fleet_latest/B11_CSVLog-2024-10-07_0312.csv', 'BDT_fleet_latest/B12_CSVLog-2024-10-24_2234.csv', 'BDT_fleet_latest/B13_CSVLog-2024-10-22_2231.csv', 'BDT_fleet_latest/B14_CSVLog-2024-06-19_0327.csv', 'BDT_fleet_latest/B15_CSVLog-2024-11-03_0218.csv', 'BDT_fleet_latest/B16_CSVLog-2024-10-29_2253.csv', 'BDT_fleet_latest/B17_CSVLog-2024-05-23_0305.csv', 'BDT_fleet_latest/B18_CSVLog-2024-05-29_0152.csv', 'BDT_fleet_latest/B19_CSVLog-2024-06-10_0054.csv', 'BDT_fleet_latest/B20_CSVLog-2

## Calculate Average and Median Voltages for Control Batteries at 200 minutes

In [16]:
# Declerations
D11_CB_200mins_voltage = []
T13_CB_200mins_voltage = []
T14_CB_200mins_voltage = []
D12_CB_200mins_voltage = []
D11_CB_values = []
T13_CB_values = []
T14_CB_values = []
D12_CB_values = []
cb_single_train_average = 0
cb_fleet_average = 0
cb_median_avg_train_voltages = []
cb_target = 12000
cb_tolerance = 30

## Time to seconds
# 200 mins to 210 mins ==> 12000 to 12600
# 211 mins to 220 mins ==> 12660 to 13200
# 221 mins to 230 mins ==> 13260 to 13800
# 231 mins to 240 mins ==> 13860 to 14400

for i, train in enumerate(latest_fleet_bdt):
    # print(i)
    temp_df = latest_fleet_bdt[i]
    temp_df_time = temp_df[temp_df.columns[0]]
    temp_df['datetime_column'] = pd.to_datetime(temp_df_time, format='mixed')
    temp_df['time_diff'] = (temp_df['datetime_column'] - temp_df['datetime_column'].iloc[0]).dt.total_seconds()
    
    # Extract final voltage at 200mins with a cb_tolerance of 1 minute
    # This is because the sampling times are different for BDTs across 41 trains
    filtered_df = temp_df[(temp_df['time_diff'] >= cb_target - cb_tolerance) & (temp_df['time_diff'] <= cb_target + cb_tolerance)]
    if not filtered_df.empty:
        D11_CB_200mins_voltage = filtered_df[cb_terminal_voltage[0]].iloc[0]
        T13_CB_200mins_voltage = filtered_df[cb_terminal_voltage[1]].iloc[0] 
        T14_CB_200mins_voltage = filtered_df[cb_terminal_voltage[2]].iloc[0] 
        D12_CB_200mins_voltage = filtered_df[cb_terminal_voltage[3]].iloc[0] 
    
        # Calculate train average
        temp_average = int(D11_CB_200mins_voltage+T13_CB_200mins_voltage+
                        T14_CB_200mins_voltage+D12_CB_200mins_voltage) / 4
        # Store train average
        cb_single_train_average += temp_average
        # Store single iterated train median
        cb_median_avg_train_voltages.append(int(temp_average))
        
        # Append values to lists for median calculation
        # These are the raw bank values for each car for each train
        D11_CB_values.append(int(D11_CB_200mins_voltage))
        T13_CB_values.append(int(T13_CB_200mins_voltage))
        T14_CB_values.append(int(T14_CB_200mins_voltage))
        D12_CB_values.append(int(D12_CB_200mins_voltage))
        
        # Debug
        print(f"Avergae of train B{i+1} is {temp_average/10}")
        # print(f"Running average: {cb_single_train_average/(i+1)}")
    else:
        print(f"Not looking good for you, mate Look at train B{i+1}")


## Calc Bank Median 
cb_median = []
cb_median.append(np.median(D11_CB_values))
cb_median.append(np.median(T13_CB_values))
cb_median.append(np.median(T14_CB_values))
cb_median.append(np.median(D12_CB_values))

num_trains = len(latest_fleet_bdt)
cb_fleet_average = cb_single_train_average / num_trains
print(f'CB Fleet Average is {cb_fleet_average/10}')
print(f'CB Set Median is {np.median(cb_median_avg_train_voltages)/10}')
print(f'CB Fleet Bank Median is {np.median(cb_median)/10}')

Avergae of train B1 is 97.3
Avergae of train B2 is 96.8
Avergae of train B3 is 97.875
Avergae of train B4 is 97.375
Avergae of train B5 is 97.125
Avergae of train B6 is 97.35
Avergae of train B7 is 98.025
Avergae of train B8 is 97.675
Avergae of train B9 is 96.525
Avergae of train B10 is 96.625
Avergae of train B11 is 97.575
Avergae of train B12 is 97.225
Avergae of train B13 is 97.8
Avergae of train B14 is 97.225
Avergae of train B15 is 97.325
Avergae of train B16 is 97.6
Avergae of train B17 is 96.75
Avergae of train B18 is 96.725
Avergae of train B19 is 97.475
Avergae of train B20 is 97.15
Avergae of train B21 is 97.225
Avergae of train B22 is 96.85
Avergae of train B23 is 97.45
Avergae of train B24 is 98.25
Avergae of train B25 is 97.725
Avergae of train B26 is 97.25
Avergae of train B27 is 97.55
Avergae of train B28 is 97.45
Avergae of train B29 is 97.175
Avergae of train B30 is 97.55
Avergae of train B31 is 97.65
Avergae of train B32 is 97.875
Avergae of train B33 is 97.925
Averg

## Calculate Average and Median Voltages for Lighting Batteries at 200 minutes

In [17]:
D11_LB_values = []
N17_LB_values = []
N19_LB_values = []
T13_LB_values = []
T14_LB_values = []
N18_LB_values = []
N16_LB_values = []
D12_LB_values = []

D11_LB_200mins_voltage = []
N17_LB_200mins_voltage = []
N19_LB_200mins_voltage = []
T13_LB_200mins_voltage = []
T14_LB_200mins_voltage = []
N18_LB_200mins_voltage = []
N16_LB_200mins_voltage = []
D12_LB_200mins_voltage = []

lb_single_train_average = 0
lb_fleet_average = 0
lb_median_avg_train_voltages = []
lb_target = 12000
lb_tolerance = 30

for i, train in enumerate(latest_fleet_bdt):
    # print(i)
    temp_df = latest_fleet_bdt[i]
    temp_df_time = temp_df[temp_df.columns[0]]
    temp_df['datetime_column'] = pd.to_datetime(temp_df_time, format='mixed')
    temp_df['time_diff'] = (temp_df['datetime_column'] - temp_df['datetime_column'].iloc[0]).dt.total_seconds()
    
    # Extract final voltage at 200mins with a cb_tolerance of 1 minute
    # This is because the sampling times are different for BDTs across 41 trains
    filtered_df = temp_df[(temp_df['time_diff'] >= lb_target - lb_tolerance) & (temp_df['time_diff'] <= lb_target + lb_tolerance)]
    if not filtered_df.empty:
        D11_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
        N17_LB_200mins_voltage = filtered_df[lb_terminal_voltage[1]].iloc[0]
        N19_LB_200mins_voltage = filtered_df[lb_terminal_voltage[2]].iloc[0]
        T13_LB_200mins_voltage = filtered_df[lb_terminal_voltage[3]].iloc[0]
        T14_LB_200mins_voltage = filtered_df[lb_terminal_voltage[4]].iloc[0]
        N18_LB_200mins_voltage = filtered_df[lb_terminal_voltage[5]].iloc[0]
        N16_LB_200mins_voltage = filtered_df[lb_terminal_voltage[6]].iloc[0]
        D12_LB_200mins_voltage = filtered_df[lb_terminal_voltage[7]].iloc[0]
    
        # Calculate train average
        temp_average = int(D11_LB_200mins_voltage+N17_LB_200mins_voltage+
                        N19_LB_200mins_voltage+T13_LB_200mins_voltage+
                        T14_LB_200mins_voltage+N18_LB_200mins_voltage+
                        N16_LB_200mins_voltage+D12_LB_200mins_voltage) / 8
        # Store train average
        lb_single_train_average += temp_average
        # Store single iterated train median
        lb_median_avg_train_voltages.append(int(temp_average))
        
        # Append values to lists for median calculation
        # These are the raw bank values for each car for each train
        D11_LB_values.append(int(D11_LB_200mins_voltage))
        N17_LB_values.append(int(N17_LB_200mins_voltage))
        N19_LB_values.append(int(N19_LB_200mins_voltage))
        T13_LB_values.append(int(T13_LB_200mins_voltage))
        T14_LB_values.append(int(T14_LB_200mins_voltage))
        N18_LB_values.append(int(N18_LB_200mins_voltage))
        N16_LB_values.append(int(N16_LB_200mins_voltage))
        D12_LB_values.append(int(D12_LB_200mins_voltage))
        
        # Debug
        print(f"Average of train {i+1} is {temp_average/10}")
        # print(f"Running average: {cb_single_train_average/(i+1)}")
    else:
        print(f"Not looking good for you, mate Look at train B{i+1}")


## Calc Bank Median 
lb_median = []
lb_median.append(np.median(D11_LB_values))
lb_median.append(np.median(N17_LB_values))
lb_median.append(np.median(N19_LB_values))
lb_median.append(np.median(T13_LB_values))
lb_median.append(np.median(T14_LB_values))
lb_median.append(np.median(N18_LB_values))
lb_median.append(np.median(N16_LB_values))
lb_median.append(np.median(D12_LB_values))

num_trains = len(latest_fleet_bdt)
lb_fleet_average = lb_single_train_average / num_trains
print(f'LB Fleet Average is {lb_fleet_average/10}')
print(f'LB Set Median is {np.median(lb_median_avg_train_voltages)/10}')
print(f'LB Fleet Bank Median is {np.median(lb_median)/10}')

Average of train 1 is 100.3125
Average of train 2 is 99.775
Average of train 3 is 100.4875
Average of train 4 is 100.0375
Average of train 5 is 99.825
Average of train 6 is 100.2875
Average of train 7 is 100.8125
Average of train 8 is 100.7375
Average of train 9 is 100.3
Average of train 10 is 100.425
Average of train 11 is 100.6125
Average of train 12 is 100.5375
Average of train 13 is 100.3375
Average of train 14 is 100.0625
Average of train 15 is 100.4
Average of train 16 is 100.275
Average of train 17 is 99.7375
Average of train 18 is 99.6875
Average of train 19 is 99.7375
Average of train 20 is 100.175
Average of train 21 is 100.1
Average of train 22 is 100.0125
Average of train 23 is 100.1875
Average of train 24 is 99.975
Average of train 25 is 100.2125
Average of train 26 is 99.925
Average of train 27 is 100.0
Average of train 28 is 100.0625
Average of train 29 is 100.225
Average of train 30 is 100.2625
Average of train 31 is 100.3125
Average of train 32 is 100.375
Average of tr

### Test only

In [13]:
test_df = latest_fleet_bdt[40]
test_df_time = test_df[test_df.columns[0]]
test_df['datetime_column'] = pd.to_datetime(test_df_time)
test_df['time_diff'] = (test_df['datetime_column'] - test_df['datetime_column'].iloc[0]).dt.total_seconds()


target = 12000
tolerance = 30
filtered_df = test_df[(test_df['time_diff'] >= target - tolerance) & (test_df['time_diff'] <= target + tolerance)]
if not filtered_df.empty:
    at200 = filtered_df['RT1BMCBBV'].iloc[0] 
    print(at200)
else:
    print("No data found in the specified time range.")

# at200 = test_df[test_df['time_diff'] == 12000]['RT1BMCBBV'].iloc[0]
# print(at200)

# test_df

973


/tmp/ipykernel_205000/1499418381.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_df['datetime_column'] = pd.to_datetime(test_df_time)
